This model works to estimate the prices based on the following features:

In [83]:
#importing modules
import tensorflow as tf
from tensorflow import keras
from keras import datasets
from keras import layers
from keras import models
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras as K
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
#from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib

In [20]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


In [46]:
data_path = "data.csv"
df = pd.read_csv(data_path)
df.head()

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
0,103378.0,for_sale,105000.0,3.0,2.0,0.12,1962661.0,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,52707.0,for_sale,80000.0,4.0,2.0,0.08,1902874.0,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,103379.0,for_sale,67000.0,2.0,1.0,0.15,1404990.0,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,31239.0,for_sale,145000.0,4.0,2.0,0.10,1947675.0,Ponce,Puerto Rico,731.0,1800.0,NaN
4,34632.0,for_sale,65000.0,6.0,2.0,0.05,331151.0,Mayaguez,Puerto Rico,680.0,NaN,NaN


In [47]:
set(df['status'])

{'for_sale', 'ready_to_build', 'sold'}

In [48]:
state_df_only = df[df['state'] == 'New Jersey']

In [49]:
set(state_df_only['status'])

{'for_sale', 'ready_to_build', 'sold'}

In [50]:
state_df_only['rent'] = np.random.randint(2500, 4001, size=len(state_df_only))

/var/folders/b8/466710_d78q31z8wgrvw2cc00000gn/T/ipykernel_23536/2258677479.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_df_only['rent'] = np.random.randint(2500, 4001, size=len(state_df_only))


In [51]:
state_df_only.head()

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date,rent
5199,39788.0,for_sale,333490.0,3.0,3.0,0.07,1901196.0,Burlington,New Jersey,8016.0,1500.0,NaN,3531
44418,16709.0,for_sale,339900.0,2.0,2.0,0.11,893593.0,Avenel,New Jersey,7001.0,1456.0,1987-05-20,2880
44419,60594.0,for_sale,305100.0,3.0,2.0,0.14,169015.0,Avenel,New Jersey,7001.0,1542.0,2022-01-27,2510
44420,15800.0,for_sale,205000.0,3.0,1.0,0.11,1060290.0,Avenel,New Jersey,7001.0,925.0,NaN,2795
44421,52665.0,for_sale,479000.0,NaN,NaN,0.11,1256287.0,Avenel,New Jersey,7001.0,NaN,2016-12-21,3370


In [52]:
set(state_df_only['status'])

{'for_sale', 'ready_to_build', 'sold'}

In [53]:
number_of_sold_records = (state_df_only['status'] == 'for_sale').sum()
number_of_sold_records

33400

In [54]:
state_df = state_df_only[state_df_only['status'] == 'for_sale']

In [55]:
len(state_df)

33400

In [56]:
set(state_df['status'])

{'for_sale'}

In [57]:
state_df.loc[state_df['status'] == 'for_sale', 'status'] = 'for_rent'

In [58]:
set(state_df['status'])

{'for_rent'}

In [59]:
state_df_duplicate = pd.concat([state_df,state_df,state_df,state_df,state_df,state_df,state_df,state_df], ignore_index=True)


In [60]:
len(state_df_duplicate)

267200

In [61]:
state_df_cleaned= state_df_duplicate.dropna()

In [62]:
len(state_df_cleaned)

76048

In [63]:
state_df_cleaned.head()

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date,rent
1,16709.0,for_rent,339900.0,2.0,2.0,0.11,893593.0,Avenel,New Jersey,7001.0,1456.0,1987-05-20,2880
2,60594.0,for_rent,305100.0,3.0,2.0,0.14,169015.0,Avenel,New Jersey,7001.0,1542.0,2022-01-27,2510
5,53550.0,for_rent,325000.0,3.0,3.0,0.15,1354688.0,Avenel,New Jersey,7001.0,1870.0,2002-12-17,2518
8,81259.0,for_rent,440000.0,4.0,2.0,0.17,863208.0,Avenel,New Jersey,7001.0,1842.0,2015-11-23,2614
10,48366.0,for_rent,399000.0,3.0,3.0,0.06,190536.0,Avenel,New Jersey,7001.0,1476.0,1990-02-28,3536


In [67]:
state_df_cleaned.columns

Index(['price', 'bed', 'bath', 'acre_lot', 'street', 'city', 'state',
       'zip_code', 'house_size', 'rent'],
      dtype='object')

In [65]:
#state_df_cleaned.drop(['brokered_by', 'status','prev_sold_date'], axis=1, inplace=True)
state_df_cleaned.drop(['brokered_by', 'status','prev_sold_date'], axis=1, inplace=True)

/var/folders/b8/466710_d78q31z8wgrvw2cc00000gn/T/ipykernel_23536/1652658617.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_df_cleaned.drop(['brokered_by', 'status','prev_sold_date'], axis=1, inplace=True)


In [68]:
state_df_cleaned.head()

,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,rent
1,339900.0,2.0,2.0,0.11,893593.0,Avenel,New Jersey,7001.0,1456.0,2880
2,305100.0,3.0,2.0,0.14,169015.0,Avenel,New Jersey,7001.0,1542.0,2510
5,325000.0,3.0,3.0,0.15,1354688.0,Avenel,New Jersey,7001.0,1870.0,2518
8,440000.0,4.0,2.0,0.17,863208.0,Avenel,New Jersey,7001.0,1842.0,2614
10,399000.0,3.0,3.0,0.06,190536.0,Avenel,New Jersey,7001.0,1476.0,3536


In [69]:
X=state_df_cleaned.drop(['rent'], axis=1)
y=state_df_cleaned['rent']

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [71]:
numerical_features = ['bed', 'bath', 'acre_lot', 'house_size','zip_code','street','price']
categorical_features = ['city', 'state']


In [72]:
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

In [73]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [74]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

In [75]:
X_train_preprocessed = pipeline.fit_transform(X_train)
X_test_preprocessed = pipeline.transform(X_test)

In [76]:
X_train_preprocessed.shape

(60838, 616)

In [77]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_preprocessed.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # Output layer with a single neuron for regression


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [78]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [80]:
# Train the model
history = model.fit(X_train_preprocessed, y_train, epochs=100, validation_split=0.2)


Epoch 1/100
1521/1521 ━━━━━━━━━━━━━━━━━━━━ 1s 770us/step - loss: 175479.8594 - val_loss: 175993.0000
Epoch 2/100
1521/1521 ━━━━━━━━━━━━━━━━━━━━ 1s 695us/step - loss: 174155.1562 - val_loss: 175576.4844
Epoch 3/100
1521/1521 ━━━━━━━━━━━━━━━━━━━━ 1s 681us/step - loss: 174630.4844 - val_loss: 174213.2812
Epoch 4/100
1521/1521 ━━━━━━━━━━━━━━━━━━━━ 1s 678us/step - loss: 171657.2188 - val_loss: 174750.2031
Epoch 5/100
1521/1521 ━━━━━━━━━━━━━━━━━━━━ 1s 692us/step - loss: 173159.8594 - val_loss: 174080.7656
Epoch 6/100
1521/1521 ━━━━━━━━━━━━━━━━━━━━ 1s 660us/step - loss: 172279.6094 - val_loss: 174246.6562
Epoch 7/100
1521/1521 ━━━━━━━━━━━━━━━━━━━━ 1s 669us/step - loss: 171865.6562 - val_loss: 173309.0000
Epoch 8/100
1521/1521 ━━━━━━━━━━━━━━━━━━━━ 1s 669us/step - loss: 172060.5000 - val_loss: 173489.5000
Epoch 9/100
1521/1521 ━━━━━━━━━━━━━━━━━━━━ 1s 722us/step - loss: 170752.4062 - val_loss: 172905.1250
Epoch 10/100
1521/1521 ━━━━━━━━━━━━━━━━━━━━ 1s 666us/step - loss: 171881.4688 - val_loss: 1

In [81]:
# Evaluate the model on the test set
loss = model.evaluate(X_test_preprocessed, y_test)
print(f'Test Loss: {loss}')


476/476 ━━━━━━━━━━━━━━━━━━━━ 0s 363us/step - loss: 150129.1094
Test Loss: 151255.171875


In [82]:
# Make predictions on new data
new_data = pd.DataFrame({
    'bed': [4],
    'bath': [3],
    'acre_lot': [0.8],
    'street': [893593.0],
    'city': ['Newark'],
    'state': ['New Jersey'],
    'house_size': [2200],
    'zip_code': [7003],
    'price': [500000]
})

# Preprocess the new data
new_data_preprocessed = pipeline.transform(new_data)

# Predict the price
predicted_price = model.predict(new_data_preprocessed)
print(f'Predicted Rent: {predicted_price[0][0]:.2f}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Predicted Rent: 3330.07


In [84]:
model.save('rent_prediction_model.h5')
joblib.dump(preprocessor, 'rent_preprocessor.pkl')

['preprocessor2.pkl']